# The Battle of Neighborhoods

## Week 1

##  Introduction/Business Problem

### The Problem:

What location in a given city offers the highest concentration of restaurants catering to the worlds healthiest diets?

The healthiest diets as discussed in the book "The Blue Zones Solution" by Dan Buettner defines the healthiest diets based on their effects on the body and longevity in age.

The healthiest diets are often correlated with an active lifestyle that requires daily movement. However for this exercise we are focusing just on the food aspect.

The diets tend to schedule the consumption of various foods daily, weekly and monthly.

The daily consumption usually includes; whole grains, fruit, legumes/nuts, vegetables and oils like olive oil.

The weekly consumption includes; fish & seafood, poultry, eggs and sometimes sugar sweets

The monthly consumption sometimes includes; other red meat products.

### Parameters:

The healthiest diets will be defined as;
1. Mediterranean - Greek, 
2. Mediterranean - Sardinian / Italian (ex Pizza),
3. Japanese, 
4. Pescetarian / Vegetarian

The location for the pilot will be: Toronto, Canada

A cluster will need meet the following criteria to be included in the output;

1. At least one restaurant from the four healthiest diets, unless only one cluster is found

2. No cluster can have more than 50% from one type of restaurant, promoting diversity and choice

3. The cluster radius will be limited to 1000 meters unless only one cluster is found

4. The clusters will be ranked based on cluster size-weighted restaurant likes

## The Data

The location data will be limited to Toronto, Canada.

The source of the Toronto neighborhood data will be Wikipedia: 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

The source of the Toronto Latitude and Longitude data will be: 'https://cocl.us/Geospatial_data'

The restaurant data will be sourced from FourSquare;

1. GET https://api.foursquare.com/v2/venues/search

2. GET https://api.foursquare.com/v2/venues/VENUE_ID/likes

The algorithm will start at the centre of each Toronto neighborhood and group FourSquare restaurant data into 1000 meter radius clusters.

The clusters will be merged or eliminated as the models fits to the parameters

## Week 2

In [121]:
# @hidden_cell
CLIENT_ID = 'MZCGBEIMI00WTURITIW0BD3VQCZACLIMNCWWABQTBFIQHJZB' # your Foursquare ID
CLIENT_SECRET = 'EXMBBCNCXRW0Z55KW113TZZSURFLZC5TR1X3UYASXRXB1NCD' # your Foursquare Secret
ACCESS_TOKEN = 'CGN2SY1KXLOF4NXDXWGTN343RROGRM5CNDPNIQ1X4XBIYHCE' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

In [2]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
import pandas as pd
import numpy as np
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following NEW packages will be INSTALLED:

  _anaconda_depends  pkgs/main/win-64::_anaconda_depends-5.3.1-py37_0
  altair             conda-forge/noarch::altair-4.1.0-py_1
  appdirs            conda-forge/noarch::appdirs-1.4.4-pyh9f0ad1d_0
  automat            conda-forge/noarch::automat-20.2.0-py_0
  bcrypt             conda-forge/win-64::bcrypt-3.1.7-py37h8055547_1
  blaze              pkgs/main/win-64::blaze-0.11.3-py37_0
  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  constantly         conda-forge/noarch::constantly-15.1.0-py_0
  datashape          conda-forge/noarch::datashape-0.5.4-py_1
  flask-cors         conda-forge/noarch::flask-cors-3.0.10-pyhd8ed1ab_0
  folium             conda-forge/noarch::foliu


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2019.07=py37_0
  - defaults/win-64::numba==0.44.1=py37hf9181ef_0

EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




Folium installed
Libraries imported.


# 1. Get Toronto Neighborhood data from Wikipedia

In [3]:
# Get Toronto Neighborhood data
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
print(soup.title)
from IPython.display import display_html
tab = str(soup.table)
display_html(tab,raw=True)

<title>List of postal codes of Canada: M - Wikipedia</title>


M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)
M1CScarborough(Rouge Hill / Port Union / Highland Creek),M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood)
M1EScarborough(Guildwood / Morningside / West Hill),M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned
M1HScarborough(Cedarbrae),M2HNorth York(Hillcrest Village),M3HNorth York(Bathurst Manor / Wilson Heights / Downsview North),M4HEast York(Thorncliffe Park),M5HDowntown Toronto(Richmond / Adelaide / King),M6HWest Toronto(Dufferin / Dovercourt Village),M7HNot assigned,M8HNot assigned,M9HNot assigned
M1JScarborough(Scarborough Village),M2JNorth York(Fairview / Henry Farm / Oriole),M3JNorth York(Northwood Park / York University),M4JEast YorkEast Toronto(The Danforth East),M5JDowntown Toronto(Harbourfront East / Union Station / Toronto Islands),M6JWest Toronto(Little Portugal / Trinity),M7JNot assigned,M8JNot assigned,M9JNot assigned
M1KScarborough(Kennedy Park / Ionview / East Birchmount Park),M2KNorth York(Bayview Village),M3KNorth York(Downsview)East (CFB Toronto),M4KEast Toronto(The Danforth West / Riverdale),M5KDowntown Toronto(Toronto Dominion Centre / Design Exchange),M6KWest Toronto(Brockton / Parkdale Village / Exhibition Place),M7KNot assigned,M8KNot assigned,M9KNot assigned
M1LScarborough(Golden Mile / Clairlea / Oakridge),M2LNorth York(York Mills / Silver Hills),M3LNorth York(Downsview)West,M4LEast Toronto(India Bazaar / The Beaches West),M5LDowntown Toronto(Commerce Court / Victoria Hotel),M6LNorth York(North Park / Maple Leaf Park / Upwood Park),M7LNot assigned,M8LNot assigned,M9LNorth York(Humber Summit)
M1MScarborough(Cliffside / Cliffcrest / Scarborough Village West),M2MNorth York(Willowdale / Newtonbrook),M3MNorth York(Downsview)Central,M4MEast Toronto(Studio District),M5MNorth York(Bedford Park / Lawrence Manor East),M6MYork(Del Ray / Mount Dennis / Keelsdale and Silverthorn),M7MNot assigned,M8MNot assigned,M9MNorth York(Humberlea / Emery)
M1NScarborough(Birch Cliff / Cliffside West),M2NNorth York(Willowdale)South,M3NNorth York(Downsview)Northwest,M4NCentral Toronto(Lawrence Park),M5NCentral Toronto(Roselawn),M6NYork(Runnymede / The Junction North),M7NNot assigned,M8NNot assigned,M9NYork(Weston)


In [4]:
dfs = pd.read_html(tab)
df=dfs[0]
df.head()

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


# 2. Data wrangling and clean up

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)


df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

# Combining the neighbourhoods with same Postalcode
df = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)

In [6]:
df.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
# Get Latitude and Longitude data for map
lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
lat_lon.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_Lat_Long = pd.merge(df,lat_lon,on='PostalCode')
df_Lat_Long.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [51]:
# Fiilter down to Toronto Boroughs only
df_Toronto = df_Lat_Long[df_Lat_Long['Borough'].str.contains('Toronto',regex=False)]
df_Toronto.shape

(39, 5)

In [52]:
restaurants = {'Greek', 'Sardinian','Italian', 'Japanese', 'Pescetarian', 'Vegetarian'}

df_Toronto.loc[:,'Greek'] = 0
df_Toronto.loc[:,'Sardinian'] = 0
df_Toronto.loc[:,'Italian'] = 0
df_Toronto.loc[:,'Japanese'] = 0
df_Toronto.loc[:,'Pescetarian'] = 0
df_Toronto.loc[:,'Vegetarian'] = 0

df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0,0,0,0,0
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,0,0,0,0,0
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,0,0,0,0,0
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0,0,0,0,0
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,0,0,0,0,0


# 3. Get Restuarant Response and Process Data from FourSquare

In [53]:
# Get data from FourSquare and apply restaurant count of restaurant category to Toronto dataframe
for index,hood in df_Toronto.iterrows():
        for r in restaurants:
            search_query = r
            radius = 1000
            latitude = hood['Latitude']
            longitude = hood['Longitude']
            url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
            results = requests.get(url).json()
            
            # assign relevant part of JSON to venues
            venues = results['response']['venues']

            # tranform venues into a dataframe
            dataframe = json_normalize(venues)
            
            if not dataframe.empty:
                
                df_Toronto.loc[index,[search_query]] = dataframe.shape[0]
                
                
        

In [54]:
# Sum restaurant count as Total
df_Toronto['Total'] =df_Toronto[restaurants].sum(axis=1)
df_Toronto.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,0,1,1,0,0,2
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,17,0,11,20,0,0,48
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,13,0,12,20,0,0,45
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,0,0,1,0,0,1
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,9,0,7,10,0,0,26


In [57]:
sorted_df_Toronto = df_Toronto.sort_values(by='Total', ascending=False)
sorted_df_Toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,18,0,15,20,0,2,55
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,18,0,13,21,0,0,52
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,16,0,12,20,0,4,52
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,18,0,13,21,0,0,52
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,17,0,12,21,0,0,50
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,17,0,11,20,0,0,48
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,13,0,12,20,0,0,45
92,M5W,Downtown Toronto Stn A,Enclave of M5E,43.646435,-79.374846,15,0,8,13,0,0,36
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,6,0,8,15,0,0,29
80,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,8,0,10,7,0,3,28


In [59]:
# Evaluate the restaurant category share
sorted_df_Toronto.loc[:,'Greek_per'] = sorted_df_Toronto['Greek'] / sorted_df_Toronto['Total'] 
sorted_df_Toronto.loc[:,'Sardinian_per'] = sorted_df_Toronto['Sardinian'] / sorted_df_Toronto['Total'] 
sorted_df_Toronto.loc[:,'Italian_per'] = sorted_df_Toronto['Italian'] / sorted_df_Toronto['Total'] 
sorted_df_Toronto.loc[:,'Japanese_per'] = sorted_df_Toronto['Japanese'] / sorted_df_Toronto['Total'] 
sorted_df_Toronto.loc[:,'Pescetarian_per'] = sorted_df_Toronto['Pescetarian'] / sorted_df_Toronto['Total'] 
sorted_df_Toronto.loc[:,'Vegetarian_per'] = sorted_df_Toronto['Vegetarian'] / sorted_df_Toronto['Total'] 

sorted_df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total,Greek_per,Sardinian_per,Italian_per,Japanese_per,Pescetarian_per,Vegetarian_per
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,18,0,15,20,0,2,55,0.327273,0.0,0.272727,0.363636,0.0,0.036364
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,16,0,12,20,0,4,52,0.307692,0.0,0.230769,0.384615,0.0,0.076923
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,17,0,12,21,0,0,50,0.340000,0.0,0.240000,0.420000,0.0,0.000000


In [75]:
restaurants_share = {'Greek_per', 'Sardinian_per','Italian_per', 'Japanese_per', 'Pescetarian_per', 'Vegetarian_per'}
sorted_df_Toronto['Max_Share'] = sorted_df_Toronto[restaurants_share].max(axis=1)
sorted_df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total,Greek_per,Sardinian_per,Italian_per,Japanese_per,Pescetarian_per,Vegetarian_per,Max_Share
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,18,0,15,20,0,2,55,0.327273,0.0,0.272727,0.363636,0.0,0.036364,0.363636
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000,0.403846
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,16,0,12,20,0,4,52,0.307692,0.0,0.230769,0.384615,0.0,0.076923,0.384615
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000,0.403846
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,17,0,12,21,0,0,50,0.340000,0.0,0.240000,0.420000,0.0,0.000000,0.420000


In [77]:
#drop Neighborhoods that have a higher than 50% restaurant share

results_df_Toronto = sorted_df_Toronto[ sorted_df_Toronto['Max_Share'] < 0.5 ].sort_values(by='Total', ascending=False).reset_index(drop=True)
results_df_Toronto['Cluster'] = results_df_Toronto.index + 1
results_df_Toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total,Greek_per,Sardinian_per,Italian_per,Japanese_per,Pescetarian_per,Vegetarian_per,Max_Share,Cluster
0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,18,0,15,20,0,2,55,0.327273,0.0,0.272727,0.363636,0.0,0.036364,0.363636,1
1,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000,0.403846,2
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,16,0,12,20,0,4,52,0.307692,0.0,0.230769,0.384615,0.0,0.076923,0.384615,3
3,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000,0.403846,4
4,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.648198,-79.379817,17,0,12,21,0,0,50,0.340000,0.0,0.240000,0.420000,0.0,0.000000,0.420000,5


In [78]:
print(results_df_Toronto.shape)

(14, 20)


# 4. Display Results on Map

In [125]:
# create map
from folium.features import DivIcon
from IPython.display import display

latitude_Top = results_df_Toronto.loc[results_df_Toronto['Cluster'] == 1]['Latitude'].values[0]           
longitude_Top = results_df_Toronto.loc[results_df_Toronto['Cluster'] == 1]['Longitude'].values[0]

clusters = results_df_Toronto.shape[0]

map_clusters = folium.Map(location=[latitude_Top,longitude_Top],zoom_start=15)

# set color scheme for the clusters
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighborhood, cluster in zip(results_df_Toronto['Latitude'], results_df_Toronto['Longitude'], results_df_Toronto['Neighborhood'], results_df_Toronto['Cluster']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
    folium.map.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html='<div style="font-size: 24pt">%s</div>' % str(cluster),
            )
        ).add_to(map_clusters)
       
display(map_clusters)

In [124]:
# the Top 3 Toronto neighborhoods with the best range of restaurant across the healthy diets
results_df_Toronto.head(3)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Greek,Sardinian,Italian,Japanese,Pescetarian,Vegetarian,Total,Greek_per,Sardinian_per,Italian_per,Japanese_per,Pescetarian_per,Vegetarian_per,Max_Share,Cluster
0,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,18,0,15,20,0,2,55,0.327273,0.0,0.272727,0.363636,0.0,0.036364,0.363636,1
1,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.647177,-79.381576,18,0,13,21,0,0,52,0.346154,0.0,0.250000,0.403846,0.0,0.000000,0.403846,2
2,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,16,0,12,20,0,4,52,0.307692,0.0,0.230769,0.384615,0.0,0.076923,0.384615,3


# End

## Thank you